<a href="https://colab.research.google.com/github/lorenzopaoria/Smoking-detection-and-distance-analysis/blob/main/distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Find the distance between smoker and not with help of depth model

In [19]:
# Installazione delle dipendenze
!pip install torch torchvision
!pip install opencv-python
!git clone https://github.com/DepthAnything/Depth-Anything-V2.git
%cd Depth-Anything-V2
!pip install -r requirements.txt

Cloning into 'Depth-Anything-V2'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 142 (delta 45), reused 34 (delta 34), pack-reused 67 (from 2)
Receiving objects: 100% (142/142), 45.17 MiB | 14.83 MiB/s, done.
Resolving deltas: 100% (48/48), done.
/content/Depth-Anything-V2/Depth-Anything-V2/Depth-Anything-V2


In [20]:
# Import delle librerie necessarie
import torch
import cv2
import numpy as np
from depth_anything_v2.dpt import DepthAnythingV2
from dataclasses import dataclass
from typing import List, Tuple
import math
import os
import json

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
# Configurazione del dispositivo (GPU T4 su Colab)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Dispositivo in uso: {DEVICE}")

# Configurazione del modello DepthAnythingV2
model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
    'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
    'vitl': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]},
    'vitg': {'encoder': 'vitg', 'features': 384, 'out_channels': [1536, 1536, 1536, 1536]}
}

encoder = 'vits'
model = DepthAnythingV2(**model_configs[encoder])

# Caricamento dei pesi del modello
checkpoint_path = f'/content/drive/MyDrive/pth_depth_estimation_large/depth_anything_v2_{encoder}.pth'
model.load_state_dict(torch.load(checkpoint_path, map_location='cpu'))
model = model.to(DEVICE).eval()
print(f"Modello {encoder} caricato con successo su {DEVICE}")

Dispositivo in uso: cuda


<ipython-input-22-f763e90eb305>:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path, map_location='cpu'))


Modello vits caricato con successo su cuda


In [23]:
# Definizione della classe Person
@dataclass
class Person:
    x1: int
    y1: int
    x2: int
    y2: int
    is_smoking: bool
    confidence: float

In [24]:
# Funzione per calcolare il punto centrale di una bounding box
def calculate_center_point(person: Person) -> Tuple[float, float]:
    """Calcola il punto centrale di una bounding box"""
    center_x = (person.x1 + person.x2) / 2
    center_y = (person.y1 + person.y2) / 2
    return (center_x, center_y)

In [25]:
# Funzione per ridimensionare l'immagine
def resize_to_multiple_of_patch(image, patch_size=14, target_size=(512, 512)):
    """Ridimensiona l'immagine in modo che altezza e larghezza siano multipli di patch_size"""
    h, w = image.shape[:2]
    new_h = math.ceil(h / patch_size) * patch_size
    new_w = math.ceil(w / patch_size) * patch_size
    # Ridimensiona l'immagine a una dimensione più piccola per risparmiare memoria
    return cv2.resize(image, target_size)

In [26]:
# Funzione per preprocessare l'immagine per il modello di profondità
def preprocess_image_for_depth(image, target_size=(512, 512)):
    """Prepara l'immagine per il modello di profondità"""
    resized_image = resize_to_multiple_of_patch(image, target_size=target_size)
    image_tensor = torch.from_numpy(resized_image).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    return resized_image, image_tensor

In [27]:
# Funzione per calcolare la mappa di profondità
def calculate_depth_map(image_tensor, model):
    """Genera una mappa di profondità utilizzando Depth-Anything-V2"""
    with torch.no_grad():
        depth_map = model(image_tensor.to(DEVICE))
    return depth_map.squeeze().cpu().numpy()

In [28]:
# Funzione per scalare le bounding box
def scale_bounding_boxes(people, original_image, resized_image):
    """Scala le coordinate delle bounding box in base al ridimensionamento dell'immagine"""
    orig_h, orig_w = original_image.shape[:2]
    new_h, new_w = resized_image.shape[:2]

    scale_x = new_w / orig_w
    scale_y = new_h / orig_h

    scaled_people = []
    for person in people:
        scaled_person = Person(
            x1=int(person.x1 * scale_x),
            y1=int(person.y1 * scale_y),
            x2=int(person.x2 * scale_x),
            y2=int(person.y2 * scale_y),
            is_smoking=person.is_smoking,
            confidence=person.confidence
        )
        scaled_people.append(scaled_person)

    return scaled_people

In [29]:
# Funzione per calcolare la distanza 3D
def calculate_3d_distance(p1: Person, p2: Person, depth_map, focal_length: float, image_width: float):
    """Calcola la distanza 3D tra due persone utilizzando la mappa di profondità"""
    c1 = calculate_center_point(p1)
    c2 = calculate_center_point(p2)

    c1_x, c1_y = max(0, min(int(c1[0]), depth_map.shape[1]-1)), max(0, min(int(c1[1]), depth_map.shape[0]-1))
    c2_x, c2_y = max(0, min(int(c2[0]), depth_map.shape[1]-1)), max(0, min(int(c2[1]), depth_map.shape[0]-1))

    radius = 5
    p1_area = depth_map[max(0, c1_y-radius):min(depth_map.shape[0], c1_y+radius),
                        max(0, c1_x-radius):min(depth_map.shape[1], c1_x+radius)]
    p2_area = depth_map[max(0, c2_y-radius):min(depth_map.shape[0], c2_y+radius),
                        max(0, c2_x-radius):min(depth_map.shape[1], c2_x+radius)]

    depth1 = np.median(p1_area) if p1_area.size > 0 else depth_map[c1_y, c1_x]
    depth2 = np.median(p2_area) if p2_area.size > 0 else depth_map[c2_y, c2_x]

    x1 = (c1_x - image_width / 2) * depth1 / focal_length
    y1 = depth1
    x2 = (c2_x - image_width / 2) * depth2 / focal_length
    y2 = depth2

    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    scale_factor = 0.1
    distance_meters = distance * scale_factor

    return distance_meters

In [30]:
# Funzione per trovare le distanze tra fumatori e non fumatori
def find_smoker_nonsmoker_distances(people: List[Person], depth_map, focal_length: float, image_width: float) -> List[Tuple[Person, Person, float]]:
    """Trova tutte le distanze 3D tra fumatori e non fumatori"""
    smokers = [p for p in people if p.is_smoking]
    non_smokers = [p for p in people if not p.is_smoking]
    distances = []

    for smoker in smokers:
        for non_smoker in non_smokers:
            distance = calculate_3d_distance(smoker, non_smoker, depth_map, focal_length, image_width)
            distances.append((smoker, non_smoker, distance))

    return distances

In [31]:
# Funzione per caricare le detection da un file JSON
def load_detections_from_json(json_path: str) -> List[Person]:
    """Carica le detection dal file JSON e le converte in oggetti Person"""
    with open(json_path, 'r') as f:
        data = json.load(f)

    people = []
    if 'detections' in data:
        for detection in data['detections']:
            is_smoking = detection.get('class') == 2
            bbox = detection.get('bbox', [0, 0, 0, 0])
            confidence = detection.get('confidence', 0.0)

            if len(bbox) == 4:
                people.append(Person(
                    x1=int(bbox[0]),
                    y1=int(bbox[1]),
                    x2=int(bbox[2]),
                    y2=int(bbox[3]),
                    is_smoking=is_smoking,
                    confidence=confidence
                ))

    return people

In [32]:
# Funzione per processare e salvare l'immagine
def process_and_save_image(image_path: str, people: List[Person], output_dir: str, focal_length: float, model) -> bool:
    """Processa un'immagine disegnando le distanze 3D tra i centri delle bounding box"""
    os.makedirs(output_dir, exist_ok=True)

    original_image = cv2.imread(image_path)
    if original_image is None:
        print(f"Errore nel caricamento dell'immagine: {image_path}")
        return False

    try:
        # Libera la memoria della GPU prima di elaborare l'immagine
        torch.cuda.empty_cache()

        resized_image, image_tensor = preprocess_image_for_depth(original_image)
        scaled_people = scale_bounding_boxes(people, original_image, resized_image)
        depth_map = calculate_depth_map(image_tensor, model)
        depth_display = cv2.resize(depth_map, (original_image.shape[1], original_image.shape[0]))
        normalized_depth = cv2.normalize(depth_display, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        colored_depth = cv2.applyColorMap(normalized_depth, cv2.COLORMAP_INFERNO)

        distances = find_smoker_nonsmoker_distances(scaled_people, depth_map, focal_length, resized_image.shape[1])

        visualized_image = resized_image.copy()

        YELLOW = (0, 255, 255)
        BROWN = (42, 42, 165)
        RED = (0, 0, 255)
        BLUE = (255, 0, 0)
        GREEN = (0, 255, 0)

        for person in scaled_people:
            color = RED if person.is_smoking else BLUE
            cv2.rectangle(visualized_image, (int(person.x1), int(person.y1)),
                         (int(person.x2), int(person.y2)), color, 2)
            center = calculate_center_point(person)
            cv2.circle(visualized_image, (int(center[0]), int(center[1])), 5, YELLOW, -1)

            label = "Smoker" if person.is_smoking else "Non-smoker"
            conf_text = f"{person.confidence:.2f}"
            cv2.putText(visualized_image, f"{label} {conf_text}",
                       (int(person.x1), int(person.y1) - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        for smoker, non_smoker, distance in distances:
            s_center = calculate_center_point(smoker)
            ns_center = calculate_center_point(non_smoker)

            cv2.line(visualized_image,
                    (int(s_center[0]), int(s_center[1])),
                    (int(ns_center[0]), int(ns_center[1])),
                    BROWN, 2)

            mid_point = ((s_center[0] + ns_center[0]) // 2, (s_center[1] + ns_center[1]) // 2)

            text = f"{distance:.2f}m"
            (text_w, text_h), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
            cv2.rectangle(visualized_image,
                         (int(mid_point[0] - text_w/2 - 5), int(mid_point[1] - text_h - 5)),
                         (int(mid_point[0] + text_w/2 + 5), int(mid_point[1] + 5)),
                         (255, 255, 255), -1)

            cv2.putText(visualized_image, text,
                       (int(mid_point[0] - text_w/2), int(mid_point[1])),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, GREEN, 2)

        filename = os.path.basename(image_path)
        output_path = os.path.join(output_dir, f"distances_{filename}")
        cv2.imwrite(output_path, visualized_image)

        depth_output_path = os.path.join(output_dir, f"depth_{filename}")
        cv2.imwrite(depth_output_path, colored_depth)

        h, w = original_image.shape[:2]
        resized_original = cv2.resize(original_image, (w, h))
        resized_depth = cv2.resize(colored_depth, (w, h))
        resized_visualization = cv2.resize(visualized_image, (w, h))

        composite = np.hstack((resized_original, resized_depth, resized_visualization))
        composite_output_path = os.path.join(output_dir, f"composite_{filename}")
        cv2.imwrite(composite_output_path, composite)

        print(f"Elaborazione completata per {image_path}")
        return True

    except Exception as e:
        print(f"Errore durante l'elaborazione di {image_path}: {str(e)}")
        import traceback
        traceback.print_exc()
        return False

In [33]:
# Funzione principale
def main():
    base_dir = '/content/drive/MyDrive/test_trained_person'
    output_dir = '/content/drive/MyDrive/distance_img_process'
    focal_length = 1000  # Parametro da calibrare in base al focale della camera

    images_dir = os.path.join(base_dir, 'images')
    coordinates_dir = os.path.join(base_dir, 'coordinates')

    successful = 0
    failed = 0
    skipped = 0

    print(f"Elaborazione delle immagini in {images_dir}...")

    if not os.path.exists(images_dir):
        print(f"La directory delle immagini {images_dir} non esiste!")
        return
    if not os.path.exists(coordinates_dir):
        print(f"La directory delle coordinate {coordinates_dir} non esiste!")
        return

    os.makedirs(output_dir, exist_ok=True)

    image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    print(f"Trovate {len(image_files)} immagini da elaborare.")

    for i, filename in enumerate(image_files):
        print(f"[{i+1}/{len(image_files)}] Elaborazione di {filename}...")

        image_path = os.path.join(images_dir, filename)
        json_name = f"{os.path.splitext(filename)[0]}.json"
        json_path = os.path.join(coordinates_dir, json_name)

        if not os.path.exists(json_path):
            print(f"File JSON non trovato per {filename}, saltato.")
            skipped += 1
            continue

        try:
            people = load_detections_from_json(json_path)
            if not people:
                print(f"Nessuna persona rilevata in {filename}, saltato.")
                skipped += 1
                continue

            success = process_and_save_image(image_path, people, output_dir, focal_length, model)
            if success:
                successful += 1
                print(f"✅ Immagine {filename} elaborata con successo")
            else:
                failed += 1
                print(f"Errore nell'elaborazione dell'immagine {filename}")
        except Exception as e:
            print(f"Errore catastrofico nell'elaborazione di {filename}: {str(e)}")
            import traceback
            traceback.print_exc()
            failed += 1

    print(f"\n=== RIEPILOGO DELL'ELABORAZIONE ===")
    print(f"Immagini elaborate con successo: {successful}")
    print(f"Immagini non elaborate (errori): {failed}")
    print(f"Immagini saltate (file mancanti o nessuna persona): {skipped}")
    print(f"Totale immagini processate: {successful + failed} di {len(image_files)}")
    print(f"Risultati salvati in: {output_dir}")

if __name__ == "__main__":
    import traceback
    try:
        main()
    except Exception as e:
        print(f"Errore nell'esecuzione del programma: {str(e)}")
        traceback.print_exc()

Elaborazione delle immagini in /content/drive/MyDrive/test_trained_person/images...
Trovate 34 immagini da elaborare.
[1/34] Elaborazione di trained_40.jpg...
Errore durante l'elaborazione di /content/drive/MyDrive/test_trained_person/images/trained_40.jpg: Input image height 512 is not a multiple of patch height 14
Errore nell'elaborazione dell'immagine trained_40.jpg
[2/34] Elaborazione di trained_41.jpg...
Errore durante l'elaborazione di /content/drive/MyDrive/test_trained_person/images/trained_41.jpg: Input image height 512 is not a multiple of patch height 14
Errore nell'elaborazione dell'immagine trained_41.jpg
[3/34] Elaborazione di trained_42.jpg...
Errore durante l'elaborazione di /content/drive/MyDrive/test_trained_person/images/trained_42.jpg: Input image height 512 is not a multiple of patch height 14
Errore nell'elaborazione dell'immagine trained_42.jpg
[4/34] Elaborazione di trained_43.jpg...
Errore durante l'elaborazione di /content/drive/MyDrive/test_trained_person/ima

Traceback (most recent call last):
  File "<ipython-input-32-c606a8f0237a>", line 17, in process_and_save_image
    depth_map = calculate_depth_map(image_tensor, model)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-27-f7e44315a4a4>", line 5, in calculate_depth_map
    depth_map = model(image_tensor.to(DEVICE))
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/Depth-Anything-V2/depth_anything_v2/dpt.py", line 179, in forward
    features = self.pretrained.get_intermediate_layers(x, self.intermediate_layer_idx[self.encoder], return_class_token=True)
               ^

Errore durante l'elaborazione di /content/drive/MyDrive/test_trained_person/images/trained_44.jpg: Input image height 512 is not a multiple of patch height 14
Errore nell'elaborazione dell'immagine trained_44.jpg
[6/34] Elaborazione di trained_45.jpg...
Errore durante l'elaborazione di /content/drive/MyDrive/test_trained_person/images/trained_45.jpg: Input image height 512 is not a multiple of patch height 14
Errore nell'elaborazione dell'immagine trained_45.jpg
[7/34] Elaborazione di trained_46.jpg...
Errore durante l'elaborazione di /content/drive/MyDrive/test_trained_person/images/trained_46.jpg: Input image height 512 is not a multiple of patch height 14
Errore nell'elaborazione dell'immagine trained_46.jpg
[8/34] Elaborazione di trained_47.jpg...
Errore durante l'elaborazione di /content/drive/MyDrive/test_trained_person/images/trained_47.jpg: Input image height 512 is not a multiple of patch height 14
Errore nell'elaborazione dell'immagine trained_47.jpg
[9/34] Elaborazione di tr

  File "/content/Depth-Anything-V2/depth_anything_v2/dpt.py", line 179, in forward
    features = self.pretrained.get_intermediate_layers(x, self.intermediate_layer_idx[self.encoder], return_class_token=True)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/Depth-Anything-V2/depth_anything_v2/dinov2.py", line 308, in get_intermediate_layers
    outputs = self._get_intermediate_layers_not_chunked(x, n)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/Depth-Anything-V2/depth_anything_v2/dinov2.py", line 272, in _get_intermediate_layers_not_chunked
    x = self.prepare_tokens_with_masks(x)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/Depth-Anything-V2/depth_anything_v2/dinov2.py", line 214, in prepare_tokens_with_masks
    x = self.patch_embed(x)
        ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", lin

Errore durante l'elaborazione di /content/drive/MyDrive/test_trained_person/images/trained_50.jpg: Input image height 512 is not a multiple of patch height 14
Errore nell'elaborazione dell'immagine trained_50.jpg
[12/34] Elaborazione di trained_51.jpg...
Errore durante l'elaborazione di /content/drive/MyDrive/test_trained_person/images/trained_51.jpg: Input image height 512 is not a multiple of patch height 14
Errore nell'elaborazione dell'immagine trained_51.jpg
[13/34] Elaborazione di trained_52.jpg...
Errore durante l'elaborazione di /content/drive/MyDrive/test_trained_person/images/trained_52.jpg: Input image height 512 is not a multiple of patch height 14
Errore nell'elaborazione dell'immagine trained_52.jpg
[14/34] Elaborazione di trained_53.jpg...
Errore durante l'elaborazione di /content/drive/MyDrive/test_trained_person/images/trained_53.jpg: Input image height 512 is not a multiple of patch height 14
Errore nell'elaborazione dell'immagine trained_53.jpg
[15/34] Elaborazione d

...

Traceback (most recent call last):
  File "<ipython-input-32-c606a8f0237a>", line 17, in process_and_save_image
    depth_map = calculate_depth_map(image_tensor, model)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-27-f7e44315a4a4>", line 5, in calculate_depth_map
    depth_map = model(image_tensor.to(DEVICE))
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/Depth-Anything-V2/depth_anything_v2/dpt.py", line 179, in forward
    features = self.pretrained.get_intermediate_layers(x, self.intermediate_layer_idx[self.encoder], return_class_token=True)
               ^

Errore durante l'elaborazione di /content/drive/MyDrive/test_trained_person/images/trained_72.jpg: Input image height 512 is not a multiple of patch height 14
Errore nell'elaborazione dell'immagine trained_72.jpg
[34/34] Elaborazione di trained_73.jpg...


Traceback (most recent call last):
  File "<ipython-input-32-c606a8f0237a>", line 17, in process_and_save_image
    depth_map = calculate_depth_map(image_tensor, model)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-27-f7e44315a4a4>", line 5, in calculate_depth_map
    depth_map = model(image_tensor.to(DEVICE))
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/Depth-Anything-V2/depth_anything_v2/dpt.py", line 179, in forward
    features = self.pretrained.get_intermediate_layers(x, self.intermediate_layer_idx[self.encoder], return_class_token=True)
               ^

Errore durante l'elaborazione di /content/drive/MyDrive/test_trained_person/images/trained_73.jpg: Input image height 512 is not a multiple of patch height 14
Errore nell'elaborazione dell'immagine trained_73.jpg

=== RIEPILOGO DELL'ELABORAZIONE ===
Immagini elaborate con successo: 0
Immagini non elaborate (errori): 34
Immagini saltate (file mancanti o nessuna persona): 0
Totale immagini processate: 34 di 34
Risultati salvati in: /content/drive/MyDrive/distance_img_process


Traceback (most recent call last):
  File "<ipython-input-32-c606a8f0237a>", line 17, in process_and_save_image
    depth_map = calculate_depth_map(image_tensor, model)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-27-f7e44315a4a4>", line 5, in calculate_depth_map
    depth_map = model(image_tensor.to(DEVICE))
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/Depth-Anything-V2/depth_anything_v2/dpt.py", line 179, in forward
    features = self.pretrained.get_intermediate_layers(x, self.intermediate_layer_idx[self.encoder], return_class_token=True)
               ^